### 1. 模块加载


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import jieba
import glob
import copy
import time
import os

import tensorflow as tf
from keras.models import Model
from keras.layers import multiply
from keras.layers import Dense, Embedding, Input, Flatten
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint,TensorBoard
from keras.layers.wrappers import TimeDistributed
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
tmpo_path = os.path.join("..", "data", "thinking2", "predict_obj.csv")
predict_pd = pd.read_csv(tmpo_path, header=0, encoding="utf8", dtype=str,sep='\t')
tmpo_path = os.path.join("..", "data", "thinking2", "question_obj.csv")
train_pd = pd.read_csv(tmpo_path, header=0, encoding="utf8", dtype=str,sep=',')
tmpo_path = os.path.join("..", "data", "thinking2", "review_obj.csv")
dict_pd = pd.read_csv(tmpo_path, header=0, encoding="utf8", dtype=str,sep=',')

EMBEDDING_FILE = os.path.join("..", "data", "wordvector", "wiki.zh.vec")
EMBEDDING_FILE = os.path.join("..", "data", "wordvector", "crawl-300d-2M.vec")


In [5]:
print(train_pd.head(5))
print(predict_pd.head(5))
label_list = [i1 for i1 in dict_pd["_id"]]
print(label_list)
print(len(label_list))

  level mainReviewPoints reviewPoints                 text
0     1      ['A010101']  ['A010101']                  说一说
1     1      ['A010101']  ['A010101']                  说一说
2     1      ['A010101']  ['A010101']                  说一说
3     1      ['A010102']  ['A010102']  看一看，然后将篮子按照不同属性进行分类
4     1      ['A010102']  ['A010102']  看一看，然后将车子按照不同属性进行分类
                                         Description MainReviewPoints  \
0                                  如图，右边正方体的展开图为(　　)              NaN   
1                  如图，这是一个正方体的展开图，则号码2代表的面所相对的面的号码是．              NaN   
2  将一张长与宽的比为2︰1的长方形纸片按如图(1)(2)所示的方式对折，然后沿图(3)中的虚线...              NaN   
3  图中的甲、乙是否是几何体的平面展开图，先想一想，再折一折，如果是，请说出折叠后的几何体名称、...              NaN   
4  如图，有正方体纸盒，在它的三个侧面分别画有三角形、正方形和圆，现用一把剪刀沿着它的棱剪开成一...              NaN   

  ReviewPoints Level  
0          NaN     1  
1          NaN     2  
2          NaN     1  
3          NaN     2  
4          NaN     1  
['A010101', 'A010102', 'A010103', 'A010104', 'A010105', 'A010106', 'A

### 2. 目录定义查看


In [ ]:
from subprocess import check_output
path = '../data/'
inp = 'input/testproject/'
oup = 'output/testproject/'
logp = 'logs/testproject/'
modp = 'models/testproject/'

# print(check_output(["ls", path + inp]).decode("utf8"))


# 训练数据总集
data_all_file = path + inp + 'sematic_label_train.csv'

# 训练数据集
train_file = path + inp + 'train.csv'
# 测试数据集
# test_file = path + inp + 'test_alll.csv'
test_file = predict_pd

TXT_DATA_FILE = path + inp + 'sematic_train.txt'
XLSX_DATA_FILE = path + inp + 'sematic_train.xlsx'
CSV_DATA_FILE = path + inp + 'sematic_train.csv'

# 结果文件
res_file = = os.path.join("..", "data", "thinking2", "baseline.csv")
# 模型文件
model_path = path + modp + 'model_t.h5'
out_path = path + oup + 'baseline.csv'
tensor_path = path + logp + 'baseline.csv'

# model_path = './model/model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'


### 4. 数据集 训练转化

In [ ]:
# translate to 1 hot
# 去非nan
data_all = pd.read_csv(data_all_file)
data_all = data_all[~(data_all["评论内容"].isnull())]
# list_sentences_test = test["评论内容"].fillna("CVxTz").values

# 变成one hot
data_all['postive']=0
data_all['neutral']=0
data_all['negative']=0
data_all.loc[data_all['分类'] == 1, 'postive'] = 1
data_all.loc[data_all['分类'] == 0, 'neutral'] = 1
data_all.loc[data_all['分类'] == -1, 'negative'] = 1

# # 拆分测试集,训练时跳过  *******************************
# train_all = data_all.sample(frac=1).reset_index(drop=True)
# lenth_train = train_all.shape[0]
# spint = int(1*lenth_train)
# train = train_all.loc[0:spint,:]
# test = train_all.loc[spint:,:]
# train.to_csv(train_file, index=False)
# test.to_csv(test_file, index=False)
# # 拆分测试集,训练时跳过  *******************************


### 5. 开始训练

In [ ]:
# max_features = 999999
max_features = 20000
maxlen = 300
memn = 200
dropn = 0.5

# train = pd.read_csv(train_file)
train = train_pd
print(train.shape)
# test = pd.read_csv(test_file)
test = predict_pdint
print(test.shape)
for i1 in train.index:
    train.loc[i1, "text"]=" ".join(jieba.cut(train.loc[i1, "text"]))
for i1 in test.index:
    test.loc[i1, "Description"]=" ".join(jieba.cut(test.loc[i1, "Description"]))
    
list_sentences_train = train["text"].fillna("CVxTz").values
list_sentences_test = test["Description"].fillna("CVxTz").values
# list_classes = [i1 for i1 in train.columns]
list_classes = label_list
# list_classes = ["negative","neutral","postive"]
try:
    list_classes.remove("评论内容")
    list_classes.remove("id")
except Exception as e:
    pass

y = train[list_classes].values

# list_sentences_test = test["评论内容"].fillna("CVxTz").values

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)


In [ ]:
# first way attention
# def attention_3d_block(inputs):
#     #input_dim = int(inputs.shape[2])
# #     a = Permute((2, 1))(inputs)
#     x = Reshape((maxlen * ,), name='reshape_1')(x)
#     a_probs = Dense(maxlen, activation='softmax')(inputs)
# #     a_probs = Permute((2, 1), name='attention_vec')(a)
#     #output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
#     output_attention_mul = multiply([inputs, a_probs], name='attention_mul')
#     return output_attention_mul

# lstm_out = Bidirectional(LSTM(lstm_units, return_sequences=True), name='bilstm')(drop1)
# attention_mul = attention_3d_block(lstm_out)
# attention_flatten = Flatten()(attention_mul)
# drop2 = Dropout(0.3)(attention_flatten)

def get_model(memn,dropn):
    embed_size = 105
    embed_size = 300
    # 时间步 = maxlen
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    # memory units = 50
    x = Bidirectional(LSTM(memn, return_sequences=True))(x)
# #     x = GlobalMaxPool1D()(x)
# #     x = Reshape((maxlen * memn,), name='reshape_1')(x)
#     a_probs = Dense(maxlen, activation='softmax')(x)
# #     a_probs = Permute((2, 1), name='attention_vec')(a)
#     #output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
#     x = multiply([x, a_probs], name='attention_mul')

# #     x = attention_3d_block(x)
#     x = Dropout(dropn)(x)
#     x = Bidirectional(LSTM(memn, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(dropn)(x)
    x = Dense(memn, activation="relu")(x)
    x = Dropout(dropn)(x)
    x = Dense(3, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [ ]:
batch_size = 32
epochs = 100

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorb = TensorBoard(log_dir=tensor_path, histogram_freq=10, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
callbacks_list = [checkpoint, early, tensorb] #early

# model = get_model(200, 0.2)
model = get_model(memn, dropn)
print(model.summary())
# mlist = list(range(1,7))
# droplist = list(range(1,7))
# for memn in mlist:
#     for dropn in droplist:
#         model = get_model(memn*50, dropn*0.1)
#         start = time.time()
#         model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=callbacks_list)
#         end = time.time()
#         print(end-start)
#         print(memn*50, dropn*0.1)
#         print("*".center(100,"#"))


In [ ]:
start = time.time()
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=callbacks_list)
end = time.time()
print(end-start)

In [ ]:
model.load_weights(model_path)

y_test = model.predict(X_te)
# predict(self, x, batch_size=32, verbose=0)
# predict_classes(self, x, batch_size=32, verbose=1)
# predict_proba(self, x, batch_size=32, verbose=1)
# evaluate(self, x, y, batch_size=32, verbose=1, sample_weight=None)

# test_file = path + inp + 'test_alll.csv'
sample_submission = pd.read_csv(test_file)
sample_submission["分类"]=0
sample_submission["postive"]=0
sample_submission["neutral"]=0
sample_submission["negative"]=0
print(train.head())
print(sample_submission.head())
sample_submission[list_classes] = y_test
sample_submission["max"]=sample_submission[list_classes].max(axis=1)

for indexs in sample_submission.index:  
    for  i2 in list_classes:  
        if(sample_submission.loc[indexs,i2] ==sample_submission.loc[indexs,"max"]):
            sample_submission.loc[indexs,"predict"]=i2
for i1 in list_classes:
    sample_submission.rename(columns={i1: "pred_" + i1}, inplace=True)
sample_submission.to_csv(res_file, index=False)

In [ ]:
# 正确率评估
score = model.evaluate(X_t, y, batch_size=batch_size)
print(score)
test_pd = pd.read_csv(test_file)
res_pd = pd.read_csv(res_file)
total_pd=pd.concat([res_pd,test_pd], join='outer', axis=1)
total_right=0
total_num=0
for i1 in list_classes:
    tmp_obj=total_pd[total_pd[i1] == 1]
    sum_num=tmp_obj.shape[0]
    right_num=tmp_obj[tmp_obj["predict"] == i1].shape[0]
    total_right += right_num
    total_num += sum_num
    try:
        print("%s, sum_num: %d, right_num: %d, accuracy: %.3f" % (i1,sum_num,right_num,right_num/sum_num))
    except Exception as e:
        print("%s, sum_num: %d, right_num: %d, error: %s" % (i1,sum_num,right_num,str(e)))
        
print("total data, total_num: %d, total_right: %d, accuracy: %.3f" % (total_num,total_right,total_right/total_num))